In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import mlrose_hiive

# 加载数据集
data = load_iris()
X = data.data
y = data.target

# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义一个简单的 MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1, warm_start=True)

# 训练模型一次以获取初始权重
model.fit(X_train, y_train)

# 获取初始权重
initial_weights = np.concatenate([coef.flatten() for coef in model.coefs_] + [intercept.flatten() for intercept in model.intercepts_])

# 定义适应度函数
def fitness_function(weights):
    start = 0
    for i in range(len(model.coefs_)):
        end = start + model.coefs_[i].size
        model.coefs_[i] = weights[start:end].reshape(model.coefs_[i].shape)
        start = end
    for i in range(len(model.intercepts_)):
        end = start + model.intercepts_[i].size
        model.intercepts_[i] = weights[start:end].reshape(model.intercepts_[i].shape)
        start = end

    model.fit(X_train, y_train)
    predictions = model.predict(X_train)
    accuracy = np.mean(predictions == y_train)

    return -accuracy  # 使用负准确率作为优化目标，因为 mlrose_hiive 最小化适应度函数

# 定义优化问题
problem = mlrose_hiive.ContinuousOpt(length=len(initial_weights), fitness_fn=mlrose_hiive.CustomFitness(fitness_function), maximize=False, min_val=-1.0, max_val=1.0)

# 选择优化算法
algorithm = 'genetic_alg'  # 可选 'random_hill_climb', 'simulated_annealing', 'genetic_alg', 'mimic'

# 运行优化算法
if algorithm == 'random_hill_climb':
    best_state, best_fitness,_ = mlrose_hiive.random_hill_climb(problem, max_iters=1000,curve=True)
elif algorithm == 'simulated_annealing':
    best_state, best_fitness,_ = mlrose_hiive.simulated_annealing(problem, max_iters=1000,curve=True)
elif algorithm == 'genetic_alg':
    best_state, best_fitness,_ = mlrose_hiive.genetic_alg(problem, max_iters=1000,curve=True)
elif algorithm == 'mimic':
    best_state, best_fitness,_ = mlrose_hiive.mimic(problem, max_iters=1000,curve=True)
else:
    raise ValueError("Unsupported algorithm selected")

# 将优化后的权重赋值给模型
start = 0
for i in range(len(model.coefs_)):
    end = start + model.coefs_[i].size
    model.coefs_[i] = best_state[start:end].reshape(model.coefs_[i].shape)
    start = end
for i in range(len(model.intercepts_)):
    end = start + model.intercepts_[i].size
    model.intercepts_[i] = best_state[start:end].reshape(model.intercepts_[i].shape)
    start = end

# 在测试集上评估模型
predictions = model.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(f"Test accuracy: {accuracy}")

Test accuracy: 1.0
